In [1]:
#import os
#os.getcwd()

In [2]:
#currentPath=os.getcwd()
#!mkdir breeds
#os.chdir(currentPath+'/breeds')
#!kg config -g -u imtiaz4ds@gmail.com -p Data@2015 -c dog-breed-identification
#!kg download


In [3]:
#files = !ls
#for f in files:
#    !unzip {f}

In [4]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline

In [5]:
from fastai.imports import *
from fastai.transforms import *
from fastai.conv_learner import *
from fastai.model import *
from fastai.dataset import *
from fastai.sgdr import *
from fastai.plots import *

In [6]:
PATH = "data/breeds/"

In [7]:
sz=224
arch=resnet50
bs=58

In [8]:
label_csv=f'{PATH}labels.csv'
n=len(list(open(label_csv)))-1
val_idxs=get_cv_idxs(n)
n,len(val_idxs)

(10222, 2044)

In [9]:
def get_data(sz,bs):
    tfms=tfms_from_model(arch,sz,aug_tfms=transforms_side_on,max_zoom=1.1)
    data=ImageClassifierData.from_csv(PATH,'train',f'{PATH}labels.csv',test_name='test',val_idxs=val_idxs,suffix='.jpg',tfms=tfms,bs=bs,num_workers=4)
    return data if sz>300 else data.resize(340,'tmp')

In [10]:
data=get_data(sz,bs)

HBox(children=(IntProgress(value=0, max=6), HTML(value='')))

In [11]:
learn=ConvLearner.pretrained(arch,data,precompute=True)

In [12]:
learn.precompute=False

In [13]:
learn.set_data(get_data(299,bs))

HBox(children=(IntProgress(value=0, max=6), HTML(value='')))

In [14]:
learn.fit(1e-2,3,cycle_len=1,cycle_mult=2)

HBox(children=(IntProgress(value=0, description='Epoch', max=7), HTML(value='')))

epoch:   0, train_loss: 1.729160, val_loss: 1.005894, accuracy: 0.815476
epoch:   1, train_loss: 0.940342, val_loss: 0.532420, accuracy: 0.866311
epoch:   2, train_loss: 0.685561, val_loss: 0.501166, accuracy: 0.872537
epoch:   3, train_loss: 0.618164, val_loss: 0.448346, accuracy: 0.872537
epoch:   4, train_loss: 0.514430, val_loss: 0.421595, accuracy: 0.871579
epoch:   5, train_loss: 0.458087, val_loss: 0.408091, accuracy: 0.872537
epoch:   6, train_loss: 0.405532, val_loss: 0.403502, accuracy: 0.874453



In [16]:
log_preds,y=learn.TTA(is_test=True)
probs=np.exp(log_preds)


In [25]:
probs[4].shape

(10357, 120)

In [27]:
ds=pd.DataFrame(probs[4])
ds.columns=data.classes
ds.insert(0,'id',[o[5:-4] for o in data.test_ds.fnames])

In [30]:
subm=f'{PATH}subm/'
os.makedirs(subm)
ds.to_csv(f'{subm}subm.gz',compression='gzip',index=False)
FileLink(f'{subm}subm.gz')

/home/imtiaz/Desktop/fastai/courses/dl1/data/breeds/subm/subm.gz